In [39]:
!which python

/home/ubu/projects/ds-deployment/venv/bin/python


In [40]:
import requests
import json
import pandas as pd
from urllib.error import HTTPError, URLError

# Complete dataspace flow


## Consts


In [41]:
data_space_provider = "http://127.0.0.1:1200"
data_space_consumer = "http://127.0.0.1:1100"
api_endpoint = "https://jsonplaceholder.typicode.com/comments"

data_space_provider_participant_id = ""
data_space_provider_participant_token = ""
data_space_consumer_participant_id = ""
data_space_consumer_participant_token = ""
catalog_id = ""
dataset_id = ""
distribution_id = ""
data_service_id = ""
policy_id = ""
target_id = ""

cn_consumer_id = ""
cn_provider_id = ""
agreement_id = ""
transfer_consumer_id = ""
transfer_provider_id = ""
consumer_data_address = ""

json_header = {
    "Content-Type": "application/json",
}

## Participants onboarding


### Current Participants information


In [42]:
url = data_space_provider + "/api/v1/mates/me"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

{
    "participant_id": "urn:uuid:49694213-4123-4e56-b990-3ff43db51562",
    "participant_slug": "Provider",
    "participant_type": "Provider",
    "base_url": "http://127.0.0.1:1200",
    "token": null,
    "token_actions": null,
    "saved_at": "2025-06-20T12:43:42.334708",
    "last_interaction": "2025-06-20T12:43:42.334710",
    "is_me": true
}


In [43]:
url = data_space_consumer + "/api/v1/mates/me"
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
print(json.dumps(response_as_json, indent=4))

{
    "participant_id": "urn:uuid:56e7089c-f800-4c57-84bd-61f912b9aa98",
    "participant_slug": "Consumer",
    "participant_type": "Consumer",
    "base_url": "http://127.0.0.1:1100",
    "token": null,
    "token_actions": null,
    "saved_at": "2025-06-20T12:43:55.976609",
    "last_interaction": "2025-06-20T12:43:55.976611",
    "is_me": true
}


### Consumer registering in Provider via Verifiable presentation


In [44]:
url = data_space_consumer + "/auth/manual/ssi"
payload = {
    "url": data_space_provider + "/access",
    "id": "friendly_slugname_for_provider",
    "actions": "talk"
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_text = response.text
response_as_text

'openid4vp://authorize?response_type=vp_token&client_id=http%3A%2F%2Fhost.docker.internal%3A1200%2Fverify%2FDsyoXEeNcKoj&response_mode=direct_post&presentation_definition_uri=http%3A%2F%2Fhost.docker.internal%3A1200%2Fpd%2FDsyoXEeNcKoj&client_id_scheme=redirect_uri&nonce=AJOyaYun1W7g&response_uri=http%3A%2F%2Fhost.docker.internal%3A1200%2Fverify%2FDsyoXEeNcKoj'

### Provider and Consumer ids


In [45]:
url = data_space_provider + "/api/v1/mates"
response = requests.request("GET", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
data_space_consumer_participant_id = response_as_json[1]["participant_id"]
data_space_consumer_participant_token = response_as_json[1]["token"]
print(json.dumps(response_as_json, indent=4))

[
    {
        "participant_id": "urn:uuid:49694213-4123-4e56-b990-3ff43db51562",
        "participant_slug": "Provider",
        "participant_type": "Provider",
        "base_url": "http://127.0.0.1:1200",
        "token": null,
        "token_actions": null,
        "saved_at": "2025-06-20T12:43:42.334708",
        "last_interaction": "2025-06-20T12:43:42.334710",
        "is_me": true
    },
    {
        "participant_id": "urn:uuid:0df100cf-d833-469f-8820-1beb783406b9",
        "participant_slug": "rainbow_consumer",
        "participant_type": "Consumer",
        "base_url": "http://127.0.0.1:1100",
        "token": "8nlneFEBlNyf9h3Nq2XksnezgnPrVXPRRiRbU_BU14c",
        "token_actions": "talk",
        "saved_at": "2025-06-20T13:31:24.034710",
        "last_interaction": "2025-06-20T13:31:24.034714",
        "is_me": false
    }
]


In [46]:
url = data_space_consumer + "/api/v1/mates"
response = requests.request("GET", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
data_space_provider_participant_id = response_as_json[1]["participant_id"]
data_space_provider_participant_token = response_as_json[1]["token"]
print(json.dumps(response_as_json, indent=4))

[
    {
        "participant_id": "urn:uuid:56e7089c-f800-4c57-84bd-61f912b9aa98",
        "participant_slug": "Consumer",
        "participant_type": "Consumer",
        "base_url": "http://127.0.0.1:1100",
        "token": null,
        "token_actions": null,
        "saved_at": "2025-06-20T12:43:55.976609",
        "last_interaction": "2025-06-20T12:43:55.976611",
        "is_me": true
    },
    {
        "participant_id": "urn:uuid:9bebd509-cedc-4629-844a-7501ed346eeb",
        "participant_slug": "friendly_slugname_for_provider",
        "participant_type": "Provider",
        "base_url": "http://127.0.0.1:1200",
        "token": "8nlneFEBlNyf9h3Nq2XksnezgnPrVXPRRiRbU_BU14c",
        "token_actions": "talk",
        "saved_at": "2025-06-20T13:31:24.051014",
        "last_interaction": "2025-06-20T13:31:24.051018",
        "is_me": false
    }
]


## Create Catalog instances and metadata


In [70]:
url = data_space_provider + "/api/v1/catalogs"
payload = {
    "foaf:homepage": "My dataspace catalog",
    "dct:title": "My dataspace catalog",

}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
catalog_id = response_as_json["@id"]
catalog_id

'urn:uuid:78f174e5-d149-4899-9fba-2b9c19451ac4'

In [71]:
url = data_space_provider + "/api/v1/catalogs/{}/datasets".format(catalog_id)
payload = {
    "dct:title": "My dataspace dataset",
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
dataset_id = response_as_json["@id"]
dataset_id

'urn:uuid:1ddb222f-077c-436e-af21-d018dfff0036'

In [72]:
url = data_space_provider + "/api/v1/catalogs/{}/data-services".format(catalog_id)
payload = {
    "dcat:endpointURL": api_endpoint
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
data_service_id = response_as_json["@id"]
data_service_id

'urn:uuid:aff0b5fd-c72c-43b0-9be9-c3421e2e5cd3'

In [73]:
url = data_space_provider + "/api/v1/catalogs/{}/datasets/{}/distributions".format(catalog_id, dataset_id)
payload = {
    "dcat:accessService": data_service_id,
    "dct:formats": "http+pull"
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
distribution_id = response_as_json["@id"]
distribution_id

'urn:uuid:0b6d56c7-d97c-4f64-ba59-1636eb58859f'

## Create policies


In [93]:
url = data_space_provider + "/api/v1/datasets/{}/policies".format(dataset_id)
payload = {
  "permission": [
    {
      "action": "read",
      "constraint": 
      {
        "leftOperand": "did:web:healthcare-provider.example.org",
        "operator": "eq",
        "rightOperand": "user"
      }
    }
  ]
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
policy_id = response_as_json["@id"]
target_id = response_as_json["target"]
print(json.dumps(response_as_json, indent=4))

KeyError: '@id'

## Contract Negotiation


In [ ]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-request"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "offer": {
        "@id": policy_id,    # policy id
        "target": target_id, # data id
        "@type": "Offer",
        "permission": [
            {
                "action": "read",
                "constraint": [
                    {
                        "leftOperand": "did:web:healthcare-provider.example.org",
                        "operator": "eq",
                        "rightOperand": "user"
                    }
                ]
            },
            {
                "action": "analyze",
                "constraint": [
                    {
                    "leftOperand": "purpose",
                    "operator": "eq",
                    "rightOperand": "medical-research"
                    }
                ]
            },
        ],
        "obligation": [
           {
            "action": "notify",
            "constraint": [
                {
                    "leftOperand": "event",
                    "operator": "eq",
                    "rightOperand": "read"
                }
            ],
            },
           {
            "action": "delete",
            "constraint": [
                {
                "leftOperand": "duration",
                "operator": "eq",
                "rightOperand": "P30D"
                }
            ]
            }
        ],
        "prohibition": [
            {
                "action": "sell",
                "constraint": [
                    {
                        "leftOperand": "purpose",
                        "operator": "eq",
                        "rightOperand": "commercial"
                    }
                ]
            },
            {
                "action": "share",
                "constraint": [
                    {
                        "leftOperand": "purpose",
                        "operator": "neq",
                        "rightOperand": "commercial"
                    }
                ]
            },
      
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
cn_consumer_id = response_as_json["consumerPid"]
cn_provider_id = response_as_json["providerPid"]
print(json.dumps(response_as_json, indent=2))

KeyError: 'consumerPid'

In [85]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-offer"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id, # remove to test offer from provider
    "providerPid": cn_provider_id, # remove to test offer from provider
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
        "permission": [
            {
                "action": "supermegause"
            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerParticipantId": "urn:uuid:0df100cf-d833-469f-8820-1beb783406b9",
  "consumerPid": "urn:uuid:1f6bff44-5165-4fe6-95ea-faea1a0dc84e",
  "providerPid": "urn:uuid:40c3588b-6965-4927-bb78-64fb3953e774",
  "offer": {
    "@id": "urn:uuid:bddda623-3dc5-4df9-882e-ec8389ea5e74",
    "permission": [
      {
        "action": "supermegause"
      }
    ],
    "@type": "Offer",
    "target": "urn:uuid:aa7e6b1a-bf1c-4944-8997-cb33c6a1326d"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:40c3588b-6965-4927-bb78-64fb3953e774",
    "consumerPid": "urn:uuid:1f6bff44-5165-4fe6-95ea-faea1a0dc84e",
    "state": "OFFERED"
  }
}


In [54]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-request"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
    "offer": {
        "@id": policy_id,    # policy id
        "target": target_id, # data id
        "@type": "Offer",
        "permission": [
            {
                "action": "use"

            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
cn_consumer_id = response_as_json["consumerPid"]
cn_provider_id = response_as_json["providerPid"]
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:f45e2d0d-0255-4347-9b4a-8ad766d1e3b0",
  "providerPid": "urn:uuid:4c2841e0-df00-4af1-ae17-e013574a236d",
  "offer": {
    "@id": "urn:uuid:bddda623-3dc5-4df9-882e-ec8389ea5e74",
    "permission": [
      {
        "action": "use"
      }
    ],
    "@type": "Offer",
    "target": "urn:uuid:aa7e6b1a-bf1c-4944-8997-cb33c6a1326d"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:4c2841e0-df00-4af1-ae17-e013574a236d",
    "consumerPid": "urn:uuid:f45e2d0d-0255-4347-9b4a-8ad766d1e3b0",
    "state": "REQUESTED"
  }
}


In [79]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-offer"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
    "offer": {
        "@id": policy_id,
        "target": target_id,
        "@type": "Offer",
         "permission": [
            {
            "action": "read",
            "constraint": [
                {
                "leftOperand": "did:web:healthcare-provider.example.org",
                "operator": "eq",
                "rightOperand": "user"
                }
            ]
            },
            {
            "action": "analyze",
            "constraint": [
                {
                "leftOperand": "purpose",
                "operator": "eq",
                "rightOperand": "medical-research"
                }
            ]
            },
            {
            "action": "share",
            "constraint": [
                {
                "leftOperand": "recipient",
                "operator": "in",
                "rightOperand": [
                    "did:web:research.lab1.org",
                    "did:web:university.healthdept.org"
                ]
                }
            ]
            }
        ],
        "obligation": [
            {
            "action": "notify",
            "constraint": [
                {
                "leftOperand": "event",
                "operator": "eq",
                "rightOperand": "read"
                }
            ],
            "target": "https://audit.healthdata.example.org/logs"
            },
            {
            "action": "delete",
            "constraint": [
                {
                "leftOperand": "duration",
                "operator": "eq",
                "rightOperand": "P30D"
                }
            ]
            }
        ],
        "prohibition": [
            {
            "action": "sell",
            "constraint": [
                {
                "leftOperand": "purpose",
                "operator": "eq",
                "rightOperand": "commercial"
                }
            ]
            },
            {
            "action": "share",
            "constraint": [
                {
                "leftOperand": "recipient",
                "operator": "not in",
                "rightOperand": [
                    "did:web:research.lab1.org",
                    "did:web:university.healthdept.org"
                ]
                }
            ]
            }
        ]
    }
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "error": {
    "code": "400",
    "title": "BAD_REQUEST",
    "message": "JsonDataError: Failed to deserialize the JSON body into the target type: offer.permission[2].constraint[0]: data did not match any variant of untagged enum OdrlConstraint at line 1 column 786"
  }
}


In [56]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-acceptance"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:f45e2d0d-0255-4347-9b4a-8ad766d1e3b0",
  "providerPid": "urn:uuid:4c2841e0-df00-4af1-ae17-e013574a236d",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:4c2841e0-df00-4af1-ae17-e013574a236d",
    "consumerPid": "urn:uuid:f45e2d0d-0255-4347-9b4a-8ad766d1e3b0",
    "state": "ACCEPTED"
  }
}


In [57]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-agreement"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
agreement_id=response_as_json["agreement"]["@id"]
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:f45e2d0d-0255-4347-9b4a-8ad766d1e3b0",
  "providerPid": "urn:uuid:4c2841e0-df00-4af1-ae17-e013574a236d",
  "agreement": {
    "@id": "urn:uuid:0bc50ad8-972d-4123-84ff-5a2dfdad31dd",
    "permission": [
      {
        "action": "supermegause"
      }
    ],
    "@type": "Agreement",
    "target": "urn:uuid:aa7e6b1a-bf1c-4944-8997-cb33c6a1326d",
    "assigner": "urn:uuid:49694213-4123-4e56-b990-3ff43db51562",
    "assignee": "urn:uuid:0df100cf-d833-469f-8820-1beb783406b9",
    "timestamp": "2025-06-23T07:46:25.769777072+00:00"
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:4c2841e0-df00-4af1-ae17-e013574a236d",
    "consumerPid": "urn:uuid:f45e2d0d-0255-4347-9b4a-8ad766d1e3b0",
    "state": "AGREED"
  }
}


In [77]:
url = data_space_consumer + "/api/v1/negotiations/rpc/setup-verification"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "@context": [
    "https://w3id.org/dspace/2025/1/context.jsonld"
  ],
  "@type": "ContractNegotiationError",
  "providerPid": "urn:uuid:40c3588b-6965-4927-bb78-64fb3953e774",
  "consumerPid": "urn:uuid:1f6bff44-5165-4fe6-95ea-faea1a0dc84e",
  "code": "PROVIDER_INTERNAL_ERROR",
  "reason": [
    "Provider internal error",
    "{\"@context\":[\"https://w3id.org/dspace/2025/1/context.jsonld\"],\"@type\":\"ContractNegotiationError\",\"code\":\"JSON_DATA_ERROR\",\"consumerPid\":null,\"providerPid\":null,\"reason\":[\"Error from deserializing JSON: Message ContractAgreementVerificationMessage not allowed\"]}"
  ]
}


In [59]:
url = data_space_provider + "/api/v1/negotiations/rpc/setup-finalization"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerPid": cn_consumer_id,
    "providerPid": cn_provider_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "consumerPid": "urn:uuid:f45e2d0d-0255-4347-9b4a-8ad766d1e3b0",
  "providerPid": "urn:uuid:4c2841e0-df00-4af1-ae17-e013574a236d",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "ContractNegotiation",
    "providerPid": "urn:uuid:4c2841e0-df00-4af1-ae17-e013574a236d",
    "consumerPid": "urn:uuid:f45e2d0d-0255-4347-9b4a-8ad766d1e3b0",
    "state": "FINALIZED"
  }
}


## Transfer Negotiation


In [60]:
url = data_space_consumer + "/api/v1/transfers/rpc/setup-request"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "agreementId": agreement_id,
    "format": "http+pull"
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
transfer_consumer_id = response_as_json["consumerPid"]
transfer_provider_id = response_as_json["providerPid"]
callback_address = response_as_json["callbackAddress"]
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:66443903-43a6-4c6c-a087-d4d4dc1e8030",
  "consumerPid": "urn:uuid:b20e56a6-2a6c-4d91-8a97-8effeddb4044",
  "agreementId": "urn:uuid:0bc50ad8-972d-4123-84ff-5a2dfdad31dd",
  "callbackAddress": "http://127.0.0.1:1100/urn:uuid:4b19eade-45b0-4101-8561-6b3509574030",
  "format": "http+pull",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:66443903-43a6-4c6c-a087-d4d4dc1e8030",
    "consumerPid": "urn:uuid:b20e56a6-2a6c-4d91-8a97-8effeddb4044",
    "state": "REQUESTED"
  }
}


In [61]:
url = data_space_provider + "/api/v1/transfers/rpc/setup-start"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerCallbackAddress": callback_address,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()

print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:66443903-43a6-4c6c-a087-d4d4dc1e8030",
  "consumerPid": "urn:uuid:b20e56a6-2a6c-4d91-8a97-8effeddb4044",
  "dataAddress": {
    "@type": "DataAddress",
    "endpointType": "http",
    "endpoint": "http://127.0.0.1:1200/data/urn:uuid:66443903-43a6-4c6c-a087-d4d4dc1e8030",
    "endpointProperties": [
      {
        "@type": "EndpointProperty",
        "name": "authType",
        "value": ""
      },
      {
        "@type": "EndpointProperty",
        "name": "authorization",
        "value": ""
      }
    ]
  },
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:66443903-43a6-4c6c-a087-d4d4dc1e8030",
    "consumerPid": "urn:uuid:b20e56a6-2a6c-4d91-8a97-8effeddb4044",
    "state": "STARTED"
  }
}


In [62]:
url = data_space_consumer + "/api/v1/dataplane/" + transfer_consumer_id
response = requests.request("GET", url, headers=json_header)
response_as_json = response.json()
consumer_data_address = response_as_json["process_address"]["url"]
print(json.dumps(response_as_json, indent=2))

{
  "id": "urn:uuid:b20e56a6-2a6c-4d91-8a97-8effeddb4044",
  "process_direction": "PULL",
  "upstream_hop": {
    "protocol": "",
    "url": "",
    "auth_type": "",
    "auth_content": ""
  },
  "downstream_hop": {
    "protocol": "",
    "url": "",
    "auth_type": "",
    "auth_content": ""
  },
  "process_address": {
    "protocol": "http",
    "url": "http://127.0.0.1:1100/data/urn:uuid:b20e56a6-2a6c-4d91-8a97-8effeddb4044",
    "auth_type": "",
    "auth_content": ""
  },
  "created_at": "2025-06-23T07:46:26.393874",
  "updated_at": "2025-06-23T07:46:26.473375",
  "state": "STARTED"
}


In [63]:
try :
    df = pd.read_json(consumer_data_address)
    print(df)
except HTTPError as e:
    print(e)


     postId   id                                               name  \
0         1    1                       id labore ex et quam laborum   
1         1    2          quo vero reiciendis velit similique earum   
2         1    3                      odio adipisci rerum aut animi   
3         1    4                                     alias odio sit   
4         1    5              vero eaque aliquid doloribus et culpa   
..      ...  ...                                                ...   
495     100  496  et occaecati asperiores quas voluptas ipsam no...   
496     100  497             doloribus dolores ut dolores occaecati   
497     100  498                           dolores minus aut libero   
498     100  499  excepturi sunt cum a et rerum quo voluptatibus...   
499     100  500                                 ex eaque eum natus   

                            email  \
0              Eliseo@gardner.biz   
1          Jayne_Kuhic@sydney.com   
2             Nikita@garfield.biz   

In [64]:
url = data_space_consumer + "/api/v1/transfers/rpc/setup-suspension"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:66443903-43a6-4c6c-a087-d4d4dc1e8030",
  "consumerPid": "urn:uuid:b20e56a6-2a6c-4d91-8a97-8effeddb4044",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:66443903-43a6-4c6c-a087-d4d4dc1e8030",
    "consumerPid": "urn:uuid:b20e56a6-2a6c-4d91-8a97-8effeddb4044",
    "state": "SUSPENDED"
  }
}


In [65]:
try :
    df = pd.read_json(consumer_data_address)
    print(df)
except HTTPError as e:
    print(e)

HTTP Error 403: Forbidden


In [66]:
url = data_space_consumer + "/api/v1/transfers/rpc/setup-start"
payload = {
    "providerParticipantId": data_space_provider_participant_id,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:66443903-43a6-4c6c-a087-d4d4dc1e8030",
  "consumerPid": "urn:uuid:b20e56a6-2a6c-4d91-8a97-8effeddb4044",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:66443903-43a6-4c6c-a087-d4d4dc1e8030",
    "consumerPid": "urn:uuid:b20e56a6-2a6c-4d91-8a97-8effeddb4044",
    "state": "STARTED"
  }
}


In [67]:
try :
    df = pd.read_json(consumer_data_address)
    print(df)
except HTTPError as e:
    print(e)

     postId   id                                               name  \
0         1    1                       id labore ex et quam laborum   
1         1    2          quo vero reiciendis velit similique earum   
2         1    3                      odio adipisci rerum aut animi   
3         1    4                                     alias odio sit   
4         1    5              vero eaque aliquid doloribus et culpa   
..      ...  ...                                                ...   
495     100  496  et occaecati asperiores quas voluptas ipsam no...   
496     100  497             doloribus dolores ut dolores occaecati   
497     100  498                           dolores minus aut libero   
498     100  499  excepturi sunt cum a et rerum quo voluptatibus...   
499     100  500                                 ex eaque eum natus   

                            email  \
0              Eliseo@gardner.biz   
1          Jayne_Kuhic@sydney.com   
2             Nikita@garfield.biz   

In [68]:
url = data_space_provider + "/api/v1/transfers/rpc/setup-suspension"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerCallbackAddress": callback_address,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:66443903-43a6-4c6c-a087-d4d4dc1e8030",
  "consumerPid": "urn:uuid:b20e56a6-2a6c-4d91-8a97-8effeddb4044",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:66443903-43a6-4c6c-a087-d4d4dc1e8030",
    "consumerPid": "urn:uuid:b20e56a6-2a6c-4d91-8a97-8effeddb4044",
    "state": "SUSPENDED"
  }
}


In [69]:
url = data_space_provider + "/api/v1/transfers/rpc/setup-completion"
payload = {
    "consumerParticipantId": data_space_consumer_participant_id,
    "consumerCallbackAddress": callback_address,
    "providerPid": transfer_provider_id,
    "consumerPid": transfer_consumer_id,
}
response = requests.request("POST", url, headers=json_header, data=json.dumps(payload))
response_as_json = response.json()
print(json.dumps(response_as_json, indent=2))

{
  "providerPid": "urn:uuid:66443903-43a6-4c6c-a087-d4d4dc1e8030",
  "consumerPid": "urn:uuid:b20e56a6-2a6c-4d91-8a97-8effeddb4044",
  "message": {
    "@context": [
      "https://w3id.org/dspace/2025/1/context.jsonld"
    ],
    "@type": "TransferProcess",
    "providerPid": "urn:uuid:66443903-43a6-4c6c-a087-d4d4dc1e8030",
    "consumerPid": "urn:uuid:b20e56a6-2a6c-4d91-8a97-8effeddb4044",
    "state": "COMPLETED"
  }
}
